# For clarity, this addresses Issue 213 only

In [1]:
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
import pysm3.units as u
from pysm3.models.cmb import CMBMap
from pysm3.models.cmb import CMBLensed as CMBLensedOriginal

# Issue 213

When running CMBLensed with `apply_delens=True`, there is a ValueError.

In [2]:
try:
    will_crash = CMBLensedOriginal(
        nside=128,
        cmb_spectra="demo_powerspectra/c1_ps_l01.dat",
        # max_nside=None,
        # cmb_seed=0,
        apply_delens=True,
        delensing_ells="demo_powerspectra/c1_delensing_ells.dat",
        # map_dist=None,
    )
except ValueError as e:
    print(e)

operands could not be broadcast together with shapes (2201,) (2,) 


# Resolution

At line (43), adding `unpack=True`, resolves this:

In [3]:
# Revised, minimal changes to the __init__ method are all that is needed

class CMBLensedRevised(CMBMap):
    # intherit from CMBMap so we get the `get_emission` method
    def __init__(
        self,
        nside,
        cmb_spectra,
        max_nside=None,
        cmb_seed=None,
        apply_delens=False,
        delensing_ells=None,
        map_dist=None,
    ):
        """Lensed CMB

        Takes an input unlensed CMB and lensing spectrum from CAMB and uses
        Taylens to apply lensing, it optionally simulates delensing by
        suppressing the lensing power at specific scales with the user
        provided `delensing_ells`.

        Parameters
        ----------

        cmb_spectra : path
            Input text file from CAMB, spectra unlensed
        cmb_seed : int
            Numpy random seed for synfast, set to None for a random seed
        apply_delens : bool
            If true, simulate delensing with taylens
        delensing_ells : path
            Space delimited file with ells in the first columns and suppression
            factor (1 for no suppression) in the second column
        """
        try:
            super().__init__(nside=nside, max_nside=max_nside, map_dist=map_dist)
        except ValueError:
            pass  # suppress exception about not providing any input map
        self.cmb_spectra = self.read_txt(cmb_spectra, unpack=True)
        self.cmb_seed = cmb_seed
        self.apply_delens = apply_delens
        self.delensing_ells = (
            None if delensing_ells is None else self.read_txt(delensing_ells, 
                                                              unpack=True)
        )

        self.map = u.Quantity(self.run_taylens(), unit=u.uK_CMB, copy=False)


    def run_taylens(self):
        pass

CMBLensedRevised.run_taylens = CMBLensedOriginal.run_taylens

In [4]:
try:
    will_run = CMBLensedRevised(
        nside=128,
        cmb_spectra="demo_powerspectra/c1_ps.dat",
        # max_nside=None,
        # cmb_seed=0,
        apply_delens=True,
        delensing_ells="demo_powerspectra/c1_delensing_ells.dat",
        # map_dist=None,
    )
    print("Successful run.")
except ValueError as e:
    print(e)

Successful run.


Because the `c1` delensing ells are a no-op (it is a column of 1's), no change is seen when applying delensing_ells. I do not know enough about the method to evaluate success or failure. When addressing Issue 212, it needs to account for this as well.